In [7]:
# Lista de comentários fictícios do Reddit com e sem expressões de dor/problemas
comentarios_reddit = [
    # Comentários que expressam dores/problemas
    "Estou tendo dores de cabeça terríveis há semanas e nenhum remédio parece ajudar. Alguém tem alguma sugestão?",
    "Meu notebook está superaquecendo constantemente e não consigo trabalhar por mais de 30 minutos. Já perdi vários prazos por causa disso.",
    "A empresa de internet na minha região é horrível. Pago por 100Mbps e recebo menos de 10Mbps nos horários de pico.",
    "Não consigo dormir há meses. Tenho tentado de tudo, mas a insônia está afetando meu trabalho e minha vida pessoal.",
    "Estou frustrado com o preço dos aluguéis na minha cidade. É impossível encontrar algo decente que caiba no orçamento.",
    "Meu celular novo veio com defeito na bateria e a loja se recusa a trocar. Estou indignado com o atendimento ao cliente.",
    "Tenho procurado emprego há 6 meses e ainda não consegui nada. As contas estão se acumulando e não sei mais o que fazer.",
    "O aplicativo de delivery que uso constantemente aumentou as taxas e agora é quase inviável pedir comida.",
    "A qualidade dos produtos de supermercado caiu muito e os preços só aumentam. Estou pagando mais por menos.",
    "Me sinto sozinho trabalhando em home office, a falta de interação social está me deixando deprimido.",
    
    # Comentários neutros ou positivos (sem expressão de dor/problema)
    "Acabei de terminar essa série e adorei o final. Alguém mais assistiu?",
    "Recomendo esse livro para quem gosta de ficção científica, a narrativa é incrível.",
    "Visitei aquele novo restaurante no centro ontem. A comida é realmente muito boa.",
    "O último episódio do podcast foi muito informativo, aprendi várias coisas novas.",
    "Finalmente consegui montar meu setup para home office, ficou exatamente como eu queria.",
    "Completei minha coleção de mangá hoje, me sinto realizado!",
    "Ontem fui ao parque com meu cachorro e encontramos vários outros pets. Foi um dia divertido.",
    "Adoro quando o Reddit me recomenda subreddits interessantes que eu não conhecia.",
    "Alguém aqui joga esse novo jogo? Estou procurando pessoas para jogar online.",
    "Compartilhando uma foto do bolo que fiz seguindo aquela receita que postaram aqui semana passada."
]

In [2]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
import sys
import os

sys.path.insert(0, os.path.abspath('..'))
from schemas.classify_schema import ClassifySchema

from dotenv import load_dotenv
import os

# Carregar variáveis de ambiente do arquivo .env
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# <Prompt>
summary_prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                """Você é um analista profissional de comentários e posts do Reddit. 
                   Sua tarefa é analisar os comentários e posts de alguns usuários e identificar se eles expressam algum tipo de dor ou problema.
                   Quando um comentário/post expressa dor ou problema, a saída deve ser 'YES'.
                   Quando um comentário/post não expressa dor ou problema, a saída deve ser 'NO'.
        """,
            ),
            (
                "human",
                """
                Dado esse comentário/post do reddit:

                comentário: "{comentarios_reddit}"

                Me ajude a classificar se esse comentário revela algum problema do usuário/dor ou não.
        """,
            ),
        ]
    )

# <LLM>
llm = ChatOpenAI(
    model="gpt-4o-2024-08-06"
)

# <Chain>
chain = summary_prompt | llm.with_structured_output(ClassifySchema)
    

In [8]:
# Executa a cadeia de prompts
result = chain.invoke({"comentarios_reddit": comentarios_reddit[0]})
result

ClassifySchema(has_problem='YES')

In [ ]:
only_problems = [comentarios_reddit[i] for i in range(len(result.has_problem)) if result.has_problem[i] == "YES"]
only_problems

# Retorna apenas os comentários que expressam dores/problemas
only_problems

['Estou tendo dores de cabeça terríveis há semanas e nenhum remédio parece ajudar. Alguém tem alguma sugestão?',
 'Meu notebook está superaquecendo constantemente e não consigo trabalhar por mais de 30 minutos. Já perdi vários prazos por causa disso.',
 'A empresa de internet na minha região é horrível. Pago por 100Mbps e recebo menos de 10Mbps nos horários de pico.',
 'Não consigo dormir há meses. Tenho tentado de tudo, mas a insônia está afetando meu trabalho e minha vida pessoal.',
 'Estou frustrado com o preço dos aluguéis na minha cidade. É impossível encontrar algo decente que caiba no orçamento.',
 'Meu celular novo veio com defeito na bateria e a loja se recusa a trocar. Estou indignado com o atendimento ao cliente.',
 'Tenho procurado emprego há 6 meses e ainda não consegui nada. As contas estão se acumulando e não sei mais o que fazer.',
 'O aplicativo de delivery que uso constantemente aumentou as taxas e agora é quase inviável pedir comida.',
 'A qualidade dos produtos de s

## Testing with real reddit comments and posts

In [4]:
import json

with open("../data/comments/comments_and_posts.json", "r", encoding="utf-8") as file:
	comments_and_posts = json.load(file)

comments_and_posts

['What’s an industry that desperately needs younger generations to start taking over?',
 'Welcome to ',
 'Politics',
 'Government',
 'Farming. There is a generational cliff at the boomers. ',
 'ALL of them.',
 'Trades, almost Every one is aging out quicker than they are being replaced',
 'All blue collar trades, except maybe automotive repair. No one is dumb enough to be an auto tech long term.',
 'Wholesale and distribution is getting pretty old',
 'Elevators and escalators',
 'COBOL/mainframe type work',
 'Machinists. It takes a long time to become good at it, but all the manufacturing industries rely on them.',
 'The car business on the sales side. Once this clown generation of 55+ are out of the industry the experience of buying a car will become much easier, simpler, and transparent.',
 'Advertising. Too many executives are still shouting about bringing back ‘big magical ideas’ and long-form TV ads',
 'Accounting. 75% of CPAs are 55 and older. Most CPAs work with large businesses.

In [9]:
from tqdm import tqdm

only_problems = []

for comment in tqdm(comments_and_posts):
    result_for_comment = chain.invoke({"comentarios_reddit": comment})

    if result_for_comment.has_problem == "YES":
        only_problems.append(comment)
        print(f"Comentário com problema encontrado: {comment}")

# Salva os comentários que expressam dores/problemas em um novo arquivo JSON
with open("../data/comments/comments_with_problems.json", "w", encoding="utf-8") as file:
    json.dump(only_problems, file, ensure_ascii=False, indent=4)

print(f"Total de comentários com problemas: {len(only_problems)}")

  0%|          | 7/1850 [00:05<19:57,  1.54it/s]

Comentário com problema encontrado: Trades, almost Every one is aging out quicker than they are being replaced


  0%|          | 9/1850 [00:06<20:53,  1.47it/s]

Comentário com problema encontrado: Wholesale and distribution is getting pretty old


  1%|          | 13/1850 [00:09<21:37,  1.42it/s]

...

Comentário com problema encontrado: Yes I actually never wanna do it at all but im only 5 classes away from graduating switched majors twice so am not gonna switch it again. Idk what the future holds 💀


100%|█████████▉| 1849/1850 [26:31<00:00,  1.10it/s]

Comentário com problema encontrado: I HATED my brief time as a staff accountant in real estate. I enjoyed other industries quite a bit more (or was less bored at least).


100%|██████████| 1850/1850 [26:32<00:00,  1.16it/s]

Comentário com problema encontrado: I was a staff in audit for a year. Tax for 2 months. Absolutely hated it. Now I’m in investment banking, start tomorrow. Hopefully that goes better. 🤷🏻‍♂️ Goal is to just make enough money to open a coffee shop and wine and charcuterie place with my fiance/ soon to be wife. Idk how you guys do that for 20+ years. I feel especially bad for new graduates. PE taking over, expansion of foreign teams, having to redo their work on top of your own, 60-70 hour weeks during busy season and 40-50 during non busy season. Sure you can make 150 a year in 5-7 years, after giving up all of your free time for 4 months out of the year every year. But that doesn’t even buy you a house in a lot of metros anymore bigger than a 2 bed 1 bath. I’m sure accounting used to be a solid career, and still might be if you don’t want to have a life for a number of years. But I’m just not willing to give up that time to make someone else millions of dollars.
Total de comentários co

In [11]:
len(comments_and_posts), len(only_problems)

(1850, 588)